In [98]:
"""Importing basic modules for analysis"""
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sb
from sklearn.preprocessing import StandardScaler
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 20)
%matplotlib inline

In [99]:
"""Read the raw Telecom churn data in the dataframe"""
telco_churn = pd.read_csv(r'C:\Users\npatel\Desktop\Upgrad\telecom_churn_data.csv', encoding='ISO-8859-1')
print(telco_churn.head(10))
telco_churn.shape

   mobile_number  circle_id  loc_og_t2o_mou  std_og_t2o_mou  loc_ic_t2o_mou  \
0     7000842753        109             0.0             0.0             0.0   
1     7001865778        109             0.0             0.0             0.0   
2     7001625959        109             0.0             0.0             0.0   
3     7001204172        109             0.0             0.0             0.0   
4     7000142493        109             0.0             0.0             0.0   
5     7000286308        109             0.0             0.0             0.0   
6     7001051193        109             0.0             0.0             0.0   
7     7000701601        109             0.0             0.0             0.0   
8     7001524846        109             0.0             0.0             0.0   
9     7001864400        109             0.0             0.0             0.0   

  last_date_of_month_6 last_date_of_month_7 last_date_of_month_8  \
0            6/30/2014            7/31/2014            8/31/20

(99999, 226)

In [100]:
"""Fetch the relevant dataset - Prepaid recharge data for usage analysis
   Data pack recharge and Topup recharge - (data, num)
   Analyse for 3 months 6th, 7th and 8th, validate the accuracy for 9th month customer churn.
"""
recharges = ['total_rech_num_6', 'total_rech_num_7', 'total_rech_num_8', 'total_rech_num_9', 
             'total_rech_amt_6', 'total_rech_amt_7', 'total_rech_amt_8', 'total_rech_amt_9', 
             'total_rech_data_6', 'total_rech_data_7', 'total_rech_data_8', 'total_rech_data_9']

In [101]:
"""Derive the columns for the recharge done data + num"""
telco_churn['total_rech_amt_all_6'] = telco_churn['total_rech_amt_6'] +  telco_churn['total_rech_data_6']  * telco_churn['av_rech_amt_data_6']
telco_churn['total_rech_amt_all_7'] = telco_churn['total_rech_amt_7'] + telco_churn['total_rech_data_7']  * telco_churn['av_rech_amt_data_7']
telco_churn['total_rech_amt_all_8'] = telco_churn['total_rech_amt_8'] + telco_churn['total_rech_data_8']  * telco_churn['av_rech_amt_data_8']
#For churn score
telco_churn['total_rech_amt_all_9'] = telco_churn['total_rech_amt_9'] + telco_churn['total_rech_data_9']  * telco_churn['av_rech_amt_data_9']
telco_churn.shape

(99999, 230)

In [102]:
"""Data Cleaning and Unwantned column Removal"""
"""Remove data with Low Variance, threshold = 2"""
good_variance_cols = [key for key, val in telco_churn.nunique().to_dict().items() if val > 2]
telco_churn = telco_churn[good_variance_cols]
telco_churn.shape

(99999, 206)

In [105]:
"""Drop columns used in derived columns"""
other_cols = ['total_rech_amt_6', 'total_rech_amt_7', 'total_rech_amt_8', 'total_rech_amt_9',
              'total_rech_data_6', 'total_rech_data_7', 'total_rech_data_8', 'total_rech_data_9', 
              'av_rech_amt_data_6', 'av_rech_amt_data_7', 'av_rech_amt_data_8', 'av_rech_amt_data_9',]
telco_churn.drop(other_cols, axis=1, inplace=True)
telco_churn.shape

KeyError: "['total_rech_data_6' 'total_rech_data_7' 'total_rech_data_8'\n 'av_rech_amt_data_6' 'av_rech_amt_data_7' 'av_rech_amt_data_8'] not found in axis"

In [103]:
"""Drop columns with missing value either na or 0 values (null cleaning)"""
"""Threshold = 25%"""
"""NOTE: skipping 9th month for Churn Score Analysis"""
misses = telco_churn.isnull().sum() * 100 / len(telco_churn)
missing_value_df = pd.DataFrame({'col': telco_churn.columns,
                                 'misses': misses})

misses_col = missing_value_df.loc[(missing_value_df["col"].str.contains('_9')==False) & 
                                (missing_value_df["misses"] > 25.0) &
                                  (missing_value_df["col"].str.contains('total_rech_amt_all')==False)]["col"]
print(misses_col)
telco_churn.drop(misses_col, axis=1, inplace=True)
telco_churn.shape

date_of_last_rech_data_6    date_of_last_rech_data_6
date_of_last_rech_data_7    date_of_last_rech_data_7
date_of_last_rech_data_8    date_of_last_rech_data_8
total_rech_data_6                  total_rech_data_6
total_rech_data_7                  total_rech_data_7
total_rech_data_8                  total_rech_data_8
max_rech_data_6                      max_rech_data_6
max_rech_data_7                      max_rech_data_7
max_rech_data_8                      max_rech_data_8
count_rech_2g_6                      count_rech_2g_6
                                      ...           
count_rech_3g_8                      count_rech_3g_8
av_rech_amt_data_6                av_rech_amt_data_6
av_rech_amt_data_7                av_rech_amt_data_7
av_rech_amt_data_8                av_rech_amt_data_8
arpu_3g_6                                  arpu_3g_6
arpu_3g_7                                  arpu_3g_7
arpu_3g_8                                  arpu_3g_8
arpu_2g_6                                  arp

(99999, 182)

In [106]:
"""Column Type and Naming Standardization"""
datatypes = telco_churn.dtypes
obj_cols = datatypes[datatypes=='object']
print(obj_cols)
#datetime.datetime conversion
for col in obj_cols.index:
    telco_churn[col] = pd.to_datetime(telco_churn[col])
telco_churn.head(1)

date_of_last_rech_6         object
date_of_last_rech_7         object
date_of_last_rech_8         object
date_of_last_rech_9         object
date_of_last_rech_data_9    object
dtype: object


,mobile_number,arpu_6,arpu_7,arpu_8,arpu_9,onnet_mou_6,onnet_mou_7,onnet_mou_8,onnet_mou_9,offnet_mou_6,offnet_mou_7,offnet_mou_8,offnet_mou_9,roam_ic_mou_6,roam_ic_mou_7,roam_ic_mou_8,roam_ic_mou_9,roam_og_mou_6,roam_og_mou_7,roam_og_mou_8,roam_og_mou_9,loc_og_t2t_mou_6,loc_og_t2t_mou_7,loc_og_t2t_mou_8,loc_og_t2t_mou_9,loc_og_t2m_mou_6,loc_og_t2m_mou_7,loc_og_t2m_mou_8,loc_og_t2m_mou_9,loc_og_t2f_mou_6,loc_og_t2f_mou_7,loc_og_t2f_mou_8,loc_og_t2f_mou_9,loc_og_t2c_mou_6,loc_og_t2c_mou_7,loc_og_t2c_mou_8,loc_og_t2c_mou_9,loc_og_mou_6,loc_og_mou_7,loc_og_mou_8,loc_og_mou_9,std_og_t2t_mou_6,std_og_t2t_mou_7,std_og_t2t_mou_8,std_og_t2t_mou_9,std_og_t2m_mou_6,std_og_t2m_mou_7,std_og_t2m_mou_8,std_og_t2m_mou_9,std_og_t2f_mou_6,...,max_rech_amt_9,date_of_last_rech_6,date_of_last_rech_7,date_of_last_rech_8,date_of_last_rech_9,last_day_rch_amt_6,last_day_rch_amt_7,last_day_rch_amt_8,last_day_rch_amt_9,date_of_last_rech_data_9,total_rech_data_9,max_rech_data_9,count_rech_2g_9,count_rech_3g_9,av_rech_amt_data_9,vol_2g_mb_6,vol_2g_mb_7,vol_2g_mb_8,vol_2g_mb_9,vol_3g_mb_6,vol_3g_mb_7,vol_3g_mb_8,vol_3g_mb_9,arpu_3g_9,arpu_2g_9,monthly_2g_6,monthly_2g_7,monthly_2g_8,monthly_2g_9,sachet_2g_6,sachet_2g_7,sachet_2g_8,sachet_2g_9,monthly_3g_6,monthly_3g_7,monthly_3g_8,monthly_3g_9,sachet_3g_6,sachet_3g_7,sachet_3g_8,sachet_3g_9,aon,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g,sep_vbc_3g,total_rech_amt_all_6,total_rech_amt_all_7,total_rech_amt_all_8,total_rech_amt_all_9
0,7000842753,197.385,214.816,213.803,21.1,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,...,0,2014-06-21,2014-07-16,2014-08-08,2014-09-28,252,252,252,0,NaT,NaN,NaN,NaN,NaN,NaN,30.13,1.32,5.75,0.0,83.57,150.76,109.61,0.0,NaN,NaN,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,968,30.4,0.0,101.2,3.58,614.0,504.0,504.0,NaN


In [107]:
"""Since the month refernace to each column is tandardised to numeric gregorian sequence,
    lets refer the months in all coumns in same fashion
    6-jun
    7-jul
    8-aug
    9-sept
"""
#Rename
telco_churn.rename(columns={'jun_vbc_3g': 'vbc_3g_6', 
                            'jul_vbc_3g': 'vbc_3g_7', 
                            'aug_vbc_3g': 'vbc_3g_8',
                            'sep_vbc_3g': 'vbc_3g_9'}, inplace=True)
telco_churn.head(1)

,mobile_number,arpu_6,arpu_7,arpu_8,arpu_9,onnet_mou_6,onnet_mou_7,onnet_mou_8,onnet_mou_9,offnet_mou_6,offnet_mou_7,offnet_mou_8,offnet_mou_9,roam_ic_mou_6,roam_ic_mou_7,roam_ic_mou_8,roam_ic_mou_9,roam_og_mou_6,roam_og_mou_7,roam_og_mou_8,roam_og_mou_9,loc_og_t2t_mou_6,loc_og_t2t_mou_7,loc_og_t2t_mou_8,loc_og_t2t_mou_9,loc_og_t2m_mou_6,loc_og_t2m_mou_7,loc_og_t2m_mou_8,loc_og_t2m_mou_9,loc_og_t2f_mou_6,loc_og_t2f_mou_7,loc_og_t2f_mou_8,loc_og_t2f_mou_9,loc_og_t2c_mou_6,loc_og_t2c_mou_7,loc_og_t2c_mou_8,loc_og_t2c_mou_9,loc_og_mou_6,loc_og_mou_7,loc_og_mou_8,loc_og_mou_9,std_og_t2t_mou_6,std_og_t2t_mou_7,std_og_t2t_mou_8,std_og_t2t_mou_9,std_og_t2m_mou_6,std_og_t2m_mou_7,std_og_t2m_mou_8,std_og_t2m_mou_9,std_og_t2f_mou_6,...,max_rech_amt_9,date_of_last_rech_6,date_of_last_rech_7,date_of_last_rech_8,date_of_last_rech_9,last_day_rch_amt_6,last_day_rch_amt_7,last_day_rch_amt_8,last_day_rch_amt_9,date_of_last_rech_data_9,total_rech_data_9,max_rech_data_9,count_rech_2g_9,count_rech_3g_9,av_rech_amt_data_9,vol_2g_mb_6,vol_2g_mb_7,vol_2g_mb_8,vol_2g_mb_9,vol_3g_mb_6,vol_3g_mb_7,vol_3g_mb_8,vol_3g_mb_9,arpu_3g_9,arpu_2g_9,monthly_2g_6,monthly_2g_7,monthly_2g_8,monthly_2g_9,sachet_2g_6,sachet_2g_7,sachet_2g_8,sachet_2g_9,monthly_3g_6,monthly_3g_7,monthly_3g_8,monthly_3g_9,sachet_3g_6,sachet_3g_7,sachet_3g_8,sachet_3g_9,aon,vbc_3g_8,vbc_3g_7,vbc_3g_6,vbc_3g_9,total_rech_amt_all_6,total_rech_amt_all_7,total_rech_amt_all_8,total_rech_amt_all_9
0,7000842753,197.385,214.816,213.803,21.1,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,...,0,2014-06-21,2014-07-16,2014-08-08,2014-09-28,252,252,252,0,NaT,NaN,NaN,NaN,NaN,NaN,30.13,1.32,5.75,0.0,83.57,150.76,109.61,0.0,NaN,NaN,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,968,30.4,0.0,101.2,3.58,614.0,504.0,504.0,NaN


In [108]:
"""Drop Highly Correlated Columns for dimension reduction and simplicity"""
# Create correlation matrix
corr = telco_churn.corr().abs()

# Select upper triangle of correlation matrix
upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(np.bool))
corr =  corr.stack()
# Find index of feature columns with correlation greater than 0.95
high_corr_drop_cols = [column for column in upper.columns if any(upper[column] > 0.65)]
#high_corr_drop_cols
high_corr_drop_cols = corr[(corr > 0.65) & (corr != 1.0)].sort_values()

In [109]:
corr_drop_cols = set(filter(lambda x: 'loc' in x or 'std' in x, list(high_corr_drop_cols.reset_index()['level_0'])))
print(corr_drop_cols, len(corr_drop_cols))
telco_churn.drop(corr_drop_cols, axis=1, inplace=True)
telco_churn.shape

{'std_ic_t2t_mou_6', 'std_og_mou_8', 'loc_og_t2t_mou_9', 'std_ic_t2f_mou_7', 'std_ic_mou_9', 'std_og_t2t_mou_8', 'loc_ic_t2f_mou_6', 'std_ic_t2t_mou_7', 'loc_og_mou_6', 'loc_ic_t2m_mou_8', 'std_ic_t2m_mou_6', 'loc_ic_t2f_mou_7', 'std_ic_t2m_mou_9', 'std_og_mou_7', 'std_og_t2t_mou_9', 'std_og_t2f_mou_8', 'loc_ic_mou_6', 'loc_ic_t2f_mou_9', 'loc_og_mou_7', 'loc_ic_mou_7', 'std_og_mou_6', 'std_og_t2m_mou_8', 'loc_ic_t2t_mou_8', 'loc_ic_t2m_mou_7', 'std_og_t2t_mou_7', 'loc_ic_mou_8', 'loc_og_t2t_mou_8', 'std_og_mou_9', 'loc_ic_t2m_mou_9', 'loc_ic_t2f_mou_8', 'std_og_t2m_mou_7', 'loc_og_t2f_mou_6', 'loc_og_t2m_mou_8', 'std_ic_mou_6', 'std_og_t2t_mou_6', 'std_ic_t2m_mou_8', 'loc_og_t2f_mou_9', 'loc_og_t2t_mou_6', 'std_ic_t2t_mou_8', 'loc_og_t2f_mou_7', 'loc_ic_t2m_mou_6', 'loc_og_t2m_mou_7', 'std_ic_mou_8', 'loc_ic_mou_9', 'std_og_t2m_mou_9', 'std_og_t2f_mou_7', 'std_og_t2f_mou_6', 'std_ic_t2t_mou_9', 'std_ic_t2f_mou_8', 'std_ic_mou_7', 'std_og_t2f_mou_9', 'std_ic_t2f_mou_9', 'loc_og_t2m_mou

(99999, 118)

In [110]:
telco_churn.head()
"""Lets get high-value customers using Prepaid Services"""
"""As stated in the Problem statement Those who have recharged with an amount more than or equal to X,
where X is the 70th percentile of the average recharge amount in the first two months are high-value customers"""

avg_amt = (telco_churn["total_rech_amt_all_6"].fillna(0) + telco_churn["total_rech_amt_all_7"].fillna(0))/2.0

#70th Percentile of Avg Recharge Amount
percentile_amt_70 = np.percentile(avg_amt, 70.0)
print('70th percentile of the average recharge amount in the first two months are high-value customers: ', percentile_amt_70)

main_cust_churn = telco_churn[avg_amt >= percentile_amt_70]
print(main_cust_churn.shape)

70th percentile of the average recharge amount in the first two months are high-value customers:  121.79999999999563
(30000, 118)


In [ ]:
"""Now in order to evaluate our Prediction accuracy, we will define the Dependent Parameter
   CHURN, which is denoted using:
   1 - Customer churns
   0 - Customer does not churns
   For 9th month(Septemner)
"""
